# Caseload Modelling - Monthly Totals Cases Opened


* Cases classified as 'Ignored Cases' have been removed in this analysis.  
* Adjudicated cases have not been removed in order to include this feature in some analysis.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hideinput
import pandas as pd
import numpy  as np
from datetime import datetime
import altair as alt

from vega_datasets import data
_=alt.data_transformers.disable_max_rows()

#year_portion_limit = 0.8

debug = 1

CESSATION_CONTINUATION = {
    'Driver Deceased': 'Deceased', 
    'Drivers found medically unfit to drive':'Cessation',
    'Drivers that did not respond; cancelled license':'Cessation',
    'Drivers that voluntarily surrendered their license':'Cessation',
    'Drivers ultimately found fit to drive':'Continuation',
    'Cases remaining open at time of reporting':'Continuation'
}



In [ ]:
#hide
%reload_ext autoreload
from cheeky_monkey import *

data_file_path = r'C:\Users\mbeaulieu\rsi_project_book\Data\\'
cases2018_df = get_2018processed_case_data(data_file_path)



# cases_df = pd.read_csv(r'.\data\cases_processed.csv',parse_dates=['BIRTHDATE', 'CASE_OPENED_DT','PREV_CASE_END_DT','LAST_STATUS_DATE'], dtype={'DRIVERS_LICENSE_NO': str})
# cases_df = cases_df[(cases_df['Ignore Case'] == 0) ]

# adjudicated_cases_df = cases_df[(cases_df['Is Adjudicated'] == 'Adjudicated') ]

# #cases2019_df = cases_df[cases_df.opened_year == 2019].reset_index()

# cases_df['Age Category'] = cases_df.apply( lambda x: 'Over 80 ' if x.age_bucket >= 80 else 'Under 80', axis=1)
# cases_df['Type Origin'] = cases_df.apply( lambda x: str(x['CASE_CD']) + '_' + str(x['ORIGIN_CD']), axis=1)
# cases_df['Type & Origin Desc'] = cases_df.apply( lambda x: str(x['CASE_DSC']) + ' & ' + str(x['ORIGIN_DSC']), axis=1)
# cases_df['Case Length Over 30 Days'] = cases_df.apply( lambda x: True if x['case_length_days'] >= 30 else False, axis=1)
# cases_df['Case Length Over 60 Days'] = cases_df.apply( lambda x: True if x['case_length_days'] >= 60 else False, axis=1)
# cases_df['Is Backlog'] = cases_df.apply( lambda x: False if (x['Case Length Over 30 Days'] == False or x['GENERAL_STATUS'] == 'Closed' ) else True, axis=1)





In [ ]:
# caseload_data_by_month = pd.read_csv(r'.\data\generated\caseloaddata_by_month.csv',parse_dates=['CASE_OPENED_DT'])
# caseload_data_by_month['Opened Month'] = caseload_data_by_month.apply(lambda x: x['CASE_OPENED_DT'].strftime('%b') + '-' + x['CASE_OPENED_DT'].strftime('%Y'), axis=1)

generated_file_path = r'C:\Users\mbeaulieu\rsi_project_book\Data\Generated\\'

caseload_data_by_month = get_generated_caseload_data_bymonth(generated_file_path)

## 4 Years of case data - Jan 2018 to Oct 2021

In [ ]:
#from altair_saver import save
aggregation = {
         'Open Count': ('Open Count','sum'),

   'Total Cases': ('Open Count','sum'),
}
legend_order = caseload_data_by_month['Opened Month'].unique()
backlog_data_byday = caseload_data_by_month.groupby(['Year Span', 'Age Category','CASE_OPENED_DT','Opened Month']) \
.agg(** aggregation).reset_index()

montly_backlog_all = alt.Chart(backlog_data_byday).mark_area().encode(
    y = alt.Y("sum(Total Cases):Q" ),
    x = alt.X("Year Span:T",  scale=alt.Scale(zero=False) ),
   color = alt.Color("Opened Month:N", sort=legend_order ),
    tooltip=['CASE_OPENED_DT:T']
).properties(
    width=700,
    height=400
)

montly_backlog_all



In [ ]:
#from altair_saver import save
aggregation = {
         'Open Count': ('Open Count','sum'),

   'Status Count': ('Status Count','sum'),
}
legend_order = caseload_data_by_month['Opened Month'].unique()
backlog_data_byday = caseload_data_by_month.groupby(['Year Span', 'Age Category','CASE_OPENED_DT','Opened Month','Status']) \
.agg(** aggregation).reset_index()

montly_backlog_all = alt.Chart(backlog_data_byday).transform_filter(
    "datum['Status'] == 'Open'"
).mark_area().encode(
    y = alt.Y("sum(Status Count):Q" ),
    x = alt.X("Year Span:T",  scale=alt.Scale(zero=False) ),
   color = alt.Color("Opened Month:N", sort=legend_order ),
    tooltip=['CASE_OPENED_DT:T']
).properties(
    width=700,
    height=400
)

montly_backlog_all



In [ ]:
# #from altair_saver import save
# aggregation = {
#          'Open Count': ('Open Count','sum'),

#    'Total Cases': ('Case Count','sum'),
# }

# row_order = caseload_data_by_month['Opened Month'].unique()
# backlog_data_byday = caseload_data_by_month.groupby(['Year Span', 'Age Category','CASE_OPENED_DT','Opened Month']) \
# .agg(** aggregation).reset_index()

# montly_backlog_all = alt.Chart(backlog_data_byday).mark_bar().encode(
#     y = alt.Y("sum(Open Count):Q" ),
#     x = alt.X("Year Span:T",  scale=alt.Scale(zero=False) ),
# #   color = "CASE_OPENED_DT:N",
#     tooltip=['CASE_OPENED_DT:T'],
#     row = alt.Row('Opened Month',sort=row_order)
# #    x=alt.X('Categories', sort=row_order),
# ).properties(
#     width=600,
#     height=100
# )

# montly_backlog_all



In [ ]:
source2 = [{
            'start': '2020-03-17T00:00:00.000000000',
            'end': '2020-08-18T00:00:00.000000000',
            'event': 'Covid Shutdown'
          },
          {            
            'start': '2020-08-19T00:00:00.000000000',
            'end': '2021-01-01T00:00:00.000000000',
            'event': 'Start up'
          }]


#source = alt.pd.DataFrame(source)
source2 = alt.pd.DataFrame(source2)

In [ ]:
#from altair_saver import save

monthly_team_capacity = 33780

aggregation = {
         'Open Count': ('Open Count','sum'),

   'Total Cases': ('Case Count','sum'),
}
legend_order = caseload_data_by_month['Opened Month'].unique()
backlog_data_byday = caseload_data_by_month.groupby(['Year Span', 'Age Category','CASE_OPENED_DT','Opened Month']) \
.agg(** aggregation).reset_index()

montly_backlog_all = alt.Chart(backlog_data_byday).transform_filter(
    "datum['Open Count'] > 0"

).mark_area().encode(
    y = alt.Y("sum(Open Count):Q", axis=alt.Axis(title='Caseload Count') ),
    x = alt.X("Year Span:T",  scale=alt.Scale(zero=False) ),
   color = alt.Color("Opened Month:N", sort=legend_order , legend=None),
    tooltip=['CASE_OPENED_DT:T']
).properties(
    width=800,
    height=400
)

source3 = [{
            'Team Capacity': monthly_team_capacity,
            'tooltip':'Montly Team Capacity'
          }]
source3 = pd.DataFrame(source3)
line = alt.Chart(source3).mark_rule(color='firebrick').encode(
    y=   alt.Y( 'Team Capacity', axis=None) ,
    size=alt.SizeValue(1),
    tooltip = ['Team Capacity']
)
# .transform_filter(
#     brush
# )

source2 = [{
            'start': '2020-03-17T00:00:00.000000000',
            'end': '2020-08-18T00:00:00.000000000',
            'event': 'Covid Shutdown'
          },
          {            
            'start': '2020-08-19T00:00:00.000000000',
            'end': '2021-01-01T00:00:00.000000000',
            'event': 'Post-Covid Start up'
          },
          {            
            'start': '2021-01-01T00:00:00.000000000',
            'end': '2021-09-01T00:00:00.000000000',
            'event': 'DMER Shutdown'
          }
]


source2 = pd.DataFrame(source2)
rect = alt.Chart(source2).mark_rect(
    opacity=.2
).encode(
    x=alt.X('start:T', axis=alt.Axis(title='') ),
    x2=alt.X2('end:T'), # , axis=alt.Axis(title='') ),
#    color='event:N',
    color=alt.Color('event:N'), # legend=None),
    tooltip='event:N'
)
(rect + montly_backlog_all)
# .resolve_axis(
#     x='independent',
#     #x2='independent' #+ line
# )


In [ ]:
#from altair_saver import save

monthly_team_capacity = 7894

aggregation = {
     'Open Count': ('Open Count','sum'),
   'Total Cases': ('Case Count','sum'),
}
#legend_order = backlogdata_by_month['Opened Month'].unique()

caseload_data_byday = caseload_data_by_month.groupby(['Year Span', 'Age Category','CASE_OPENED_DT','Opened Month']) \
.agg(** aggregation).reset_index()

montly_backlog_all = alt.Chart(caseload_data_by_month).transform_filter(
    "datum['Open Count'] > 0"

).mark_area().encode(
    y = alt.Y("sum(Open Count):Q" ),
    x = alt.X("Year Span:T",  scale=alt.Scale(zero=False) ),
   color = alt.Color("Age Category:N", sort=legend_order ),
#    tooltip=['CASE_OPENED_DT:T']
).properties(
    width=600,
    height=400
)

source3 = [{
            'Team Capacity': monthly_team_capacity,
            'tooltip':'Montly Team Capacity'
          }]
source3 = pd.DataFrame(source3)

line = alt.Chart(source3).mark_rule(color='firebrick').encode(
    y=     'Team Capacity',
    size=alt.SizeValue(1),
    tooltip = ['Team Capacity']
)


# .transform_filter(
#     brush
# )

# source2 = [{
#             'start': '2020-03-17T00:00:00.000000000',
#             'end': '2020-08-18T00:00:00.000000000',
#             'event': 'Covid Shutdown'
#           },
#           {            
#             'start': '2020-08-19T00:00:00.000000000',
#             'end': '2021-01-01T00:00:00.000000000',
#             'event': 'Post-Covid Start up'
#           },
#           {            
#             'start': '2021-01-01T00:00:00.000000000',
#             'end': '2021-09-01T00:00:00.000000000',
#             'event': 'DMER Shutdown'
#           }
# ]


# source2 = pd.DataFrame(source2)
# rect = alt.Chart(source2).mark_rect(
#     opacity=.4
# ).encode(
#     x='start:T',
#     x2='end:T',
#     color='event:N',
#     tooltip='event:N'
# #    , axis=None
# )
montly_backlog_all # +  line



In [ ]:
monthly_team_capacity = 7894



aggregation = {
         'Open Count': ('Open Count','sum'),

   'Status Count': ('Status Count','sum'),
}
legend_order = caseload_data_by_month['Opened Month'].unique()
backlog_data_byday = caseload_data_by_month.groupby(['Year Span', 'Age Category','CASE_OPENED_DT','Opened Month','Status']) \
.agg(** aggregation).reset_index()

montly_backlog_all = alt.Chart(backlog_data_byday).transform_filter(
    "datum['Status'] == 'Open'"
).mark_area().encode(
    y = alt.Y("sum(Status Count):Q" ),
    x = alt.X("Year Span:T",  scale=alt.Scale(zero=False) ),
   color = alt.Color("Opened Month:N", sort=legend_order ),
    tooltip=['CASE_OPENED_DT:T']
).properties(
    width=700,
    height=400
)

source3 = [{
            'Team Capacity': monthly_team_capacity,
            'tooltip':'Montly Team Capacity'
          }]
source3 = pd.DataFrame(source3)

line = alt.Chart(source3).mark_rule(color='firebrick').encode(
    y=     'Team Capacity',
    size=alt.SizeValue(1),
    tooltip = ['Team Capacity']
)


montly_backlog_all + line


